# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install lightgbm
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.7 MB/s eta 0:00:0000:0100:010m


In [2]:
data = pd.read_csv('autos.csv')

In [3]:
data['DateCrawled'] = pd.to_datetime(data['DateCrawled'], format='%Y-%m-%d')
data['MonthCrawled'] = data['DateCrawled'].dt.month
data['YearCrawled'] = data['DateCrawled'].dt.year

data = data.loc[data['RegistrationYear'] <= data['YearCrawled']]
data = data.drop(data.loc[(data['RegistrationYear'] == data['YearCrawled']) 
                          & (data['MonthCrawled'] < data['RegistrationMonth'])].index)

Разбила дату по месяцам, годам. Удалила строки, где дата публикации раньше, чем выпуск автомобиля.

In [4]:
data.info()
print(data.head(10))
data = data.drop(columns=
    ['DateCrawled', 'NumberOfPictures', 'PostalCode', 'LastSeen', 'DateCreated', 'RegistrationMonth'], axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334610 entries, 0 to 354368
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        334610 non-null  datetime64[ns]
 1   Price              334610 non-null  int64         
 2   VehicleType        316830 non-null  object        
 3   RegistrationYear   334610 non-null  int64         
 4   Gearbox            317124 non-null  object        
 5   Power              334610 non-null  int64         
 6   Model              317618 non-null  object        
 7   Kilometer          334610 non-null  int64         
 8   RegistrationMonth  334610 non-null  int64         
 9   FuelType           309414 non-null  object        
 10  Brand              334610 non-null  object        
 11  Repaired           271924 non-null  object        
 12  DateCreated        334610 non-null  object        
 13  NumberOfPictures   334610 non-null  int64   

Удалила лишние признаки, которые не имеют значения для модели.

In [5]:
print(data.duplicated().sum())

37519


In [6]:
data = data.drop_duplicates().reset_index(drop = True)

Проверила наличие дубликатов и удалила их

In [7]:
print(data['Price'].describe())
data = data.loc[data['Price'] > 500]

count    297091.000000
mean       4567.036689
std        4602.961863
min           0.000000
25%        1100.000000
50%        2890.000000
75%        6600.000000
max       20000.000000
Name: Price, dtype: float64


По диаграмме размаха межквартильный интервал +- 3 сигмы покрывают весь размах наблюдений. Выбросов не обнаружено. Удалила строки, где цена <500, это неправдоподобное значение или ошибка.

In [8]:
print(data['RegistrationYear'].describe())
data = data.loc[data['RegistrationYear'] < 2020]
data = data.loc[data['RegistrationYear'] > 1900]


count    262434.000000
mean       2002.755889
std          10.068875
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2007.000000
max        2016.000000
Name: RegistrationYear, dtype: float64


В годах выпуска неправдоподобные те, что больше текущего календарного года и меньше 1900, удалил.

In [9]:
print(data['Power'].describe())
data = data.loc[data['Power'] < 400]
data = data.loc[data['Power'] != 0]
data = data.loc[data['Power'] >= 50]

count    262414.000000
mean        117.918735
std         195.849573
min           0.000000
25%          75.000000
50%         110.000000
75%         150.000000
max       20000.000000
Name: Power, dtype: float64


По диаграмме размаха удалил значения > 400(результат 365, округлил) и околонулевые значения мощности (меньше 50лс).

In [10]:
print(data['Kilometer'].describe())

count    237047.000000
mean     126814.766692
std       37385.318151
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Kilometer, dtype: float64


В данном столбце все значения выглядят правдопрдобно и вписываются в диаграмму размаха.

In [11]:
VehicleType = data.pivot_table(index='VehicleType', values='Price', aggfunc='count')
print(VehicleType)

             Price
VehicleType       
bus          23295
convertible  16556
coupe        12529
other         2061
sedan        68334
small        50350
suv           9780
wagon        49281


In [12]:
Gearbox = data.pivot_table(index='Gearbox', values='Price', aggfunc='count')
print(Gearbox)

          Price
Gearbox        
auto      50520
manual   182731


In [13]:
Model = data.pivot_table(index='Model', values='Price', aggfunc='count')
print(Model)

         Price
Model         
100        274
145         21
147        421
156        414
159        184
...        ...
yaris      824
yeti       157
ypsilon    123
z_reihe    640
zafira    2104

[249 rows x 1 columns]


In [14]:
FuelType = data.pivot_table(index='FuelType', values='Price', aggfunc='count')
print(FuelType)

           Price
FuelType        
cng          427
electric      30
gasoline   75303
hybrid       186
lpg         4139
other         44
petrol    146475


In [15]:
Brand = data.pivot_table(index='Brand', values='Price', aggfunc='count')
print(Brand)

                Price
Brand                
alfa_romeo       1668
audi            21577
bmw             27126
chevrolet        1342
chrysler         1013
citroen          3566
dacia             726
daewoo            296
daihatsu          414
fiat             5672
ford            15782
honda            2030
hyundai          2729
jaguar            411
jeep              538
kia              1819
lada              142
lancia            295
land_rover        430
mazda            3884
mercedes_benz   24055
mini             2642
mitsubishi       2000
nissan           3278
opel            23482
peugeot          7595
porsche           535
renault         10376
rover             266
saab              405
seat             4732
skoda            4374
smart            3275
sonstige_autos   1416
subaru            525
suzuki           1632
toyota           3647
trabant             6
volkswagen      48994
volvo            2352


In [16]:
Repaired = data.pivot_table(index='Repaired', values='Price', aggfunc='count')
print(Repaired)

           Price
Repaired        
no        185676
yes        19705


Все категориальные признаки выглядят прадоподобно.

In [17]:
print(data.isnull().sum())

Price                   0
VehicleType          4861
RegistrationYear        0
Gearbox              3796
Power                   0
Model                8372
Kilometer               0
FuelType            10443
Brand                   0
Repaired            31666
MonthCrawled            0
YearCrawled             0
dtype: int64


In [18]:
data = data.fillna('N')

In [19]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    data.drop('Price', axis=1), data.Price, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

In [20]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=1000)

categor = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired', 'Kilometer', 'RegistrationYear']

In [21]:
pd.options.mode.chained_assignment = None

features_train[categor] = encoder.fit_transform(features_train[categor])
features_valid[categor] = encoder.transform(features_valid[categor])
features_test[categor] = encoder.transform(features_test[categor])

Считаю не важными для цены DateCrawled, NumberOfPictures, PostalCode, LastSeen, DateCreated, RegistrationMonth. Удалила эти столбцы. Столбцы с числовыми признаками заполнены полностью, а катерориальные признаки имеют пропуски, поменяла на 'N', будет отдельная категория. Применяю прямое кодирование (так перевела объекты в числа). 
Разделила данные на обучающую и тестовую выборки. 

## Обучение моделей

In [22]:
model_light = LGBMRegressor(random_state=12345)
%time model_light.fit(features_train, target_train)
%time predictions = model_light.predict(features_valid)
rmse_light = mean_squared_error(target_valid, predictions)**0.5
print('rmse_light:', rmse_light)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010657 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 602
[LightGBM] [Info] Number of data points in the train set: 142228, number of used features: 10
[LightGBM] [Info] Start training from score 5343.816597
CPU times: user 1.52 s, sys: 72.3 ms, total: 1.6 s
Wall time: 937 ms
CPU times: user 701 ms, sys: 14.9 ms, total: 716 ms
Wall time: 544 ms
rmse_light: 1681.1465853276025


Обучила модель LGBMRegressor.

In [23]:
max_depth = 0
rmse_tree = 5000
for i in range(1, 31):
    model_tree = DecisionTreeRegressor(random_state=12345, max_depth=i)
    model_tree.fit(features_train, target_train)
    predictions = model_tree.predict(features_valid)
    rmse_tree1 = mean_squared_error(target_valid, predictions)**0.5
    if rmse_tree1 < rmse_tree:
        rmse_tree = rmse_tree1
        max_depth = i
print('max_depth:', max_depth,'rmse_tree:', rmse_tree)

max_depth: 13 rmse_tree: 1924.1712065407787


In [24]:
model_tree = DecisionTreeRegressor(random_state=12345, max_depth=13)
%time model_tree.fit(features_train, target_train)
%time predictions = model_tree.predict(features_valid)
rmse_tree = mean_squared_error(target_valid, predictions)**0.5
print('rmse_tree:', rmse_tree)

CPU times: user 465 ms, sys: 14.1 ms, total: 479 ms
Wall time: 605 ms
CPU times: user 10.7 ms, sys: 493 µs, total: 11.2 ms
Wall time: 10.9 ms
rmse_tree: 1924.1712065407787


Обучила модель DecisionTreeRegressor. лучший показатель rmse был на max_depth=13

In [25]:
rmse_forest = 5000
max_depth = 0
n_estimators = 0

for i in range(1,31):
    for n in range(10,110,10):
        model_forest = RandomForestRegressor(random_state=12345,n_estimators=n, max_depth=i)
        model_forest.fit(features_train, target_train)
        predictions = model_forest.predict(features_valid)
        rmse_forest1 = mean_squared_error(target_valid, predictions)**0.5
        if rmse_forest1 < rmse_forest:
            rmse_forest = rmse_forest1
            max_depth = i
            n_estimators = n
print('max_depth:', i, 'n_estimators:', n, 'rmse_forest:', rmse_forest)

max_depth: 30 n_estimators: 100 rmse_forest: 1657.875464328077


In [26]:
model_forest = RandomForestRegressor(random_state=12345, n_estimators=100, max_depth=30)
%time model_forest.fit(features_train, target_train)
%time predictions = model_forest.predict(features_valid)
rmse_forest = mean_squared_error(target_valid, predictions)**0.5
print('rmse_forest:', rmse_forest)

CPU times: user 47.6 s, sys: 742 ms, total: 48.4 s
Wall time: 49.7 s
CPU times: user 2.05 s, sys: 24.7 ms, total: 2.08 s
Wall time: 2.09 s
rmse_forest: 1683.042846624122


Обучила модель RandomForestRegressor. лучший показатель rmse был на max_depth=30, n_estimators=100. В увеличении гиперпараметров не вижу особого смысла, это займет много времени и итоговая модель будет обучаться слишком долго.

## Анализ моделей

    Критерии, которые важны заказчику:
качество предсказания;

время обучения модели;

время предсказания модели.

Среди обученных моделей самой лучшей оказалась lightgbm.
Время обучения модели: 3.92 s
Время предсказания модели: 400 ms
rmse: 1681.1465853276025
Проверю выбранную модель на тестовой выборке.

In [27]:
predictions_test = model_light.predict(features_test)
rmse_light_test = mean_squared_error(target_test, predictions_test)**0.5
print('rmse_light_test:', rmse_light_test)

rmse_light_test: 1668.4037825082114


In [28]:
predictions_mean = pd.Series(target_train.mean(), index=target_test.index)
rmse_light = mean_squared_error(target_test, predictions_mean)**0.5
print('rmse_light_mean:', rmse_light)

rmse_light_mean: 4608.553507638853


    При сравнении с предсказаниями по среднему обучающей выборки, rmse больше на тестовой:
rmse_light_mean: 4623.619085005347,
rmse_light_test: 1668.4037825082114

    Вывод: В течении данного проекта разрабатывалась модель для предсказания средней рыночной стоимости автомобиля. Предложенные данные были обработаны и проанализированы. Протестировалось несколько моделей с разными гиперпараметрами и была выбрана одна с самым высоким качеством предсказания и низкими скоростью обучения и предсказания. Модель была протестирована на тестовых данных и проверена на адекватность. Лучшей моделью оказалась lightgbm. Вреия обучения - 3.92 s, время предсказания - 400 ms, rmse - 1681.15.